In [1]:
import pandas as pd
import numpy as np
import theano.tensor as T
import theano
import theano.gradient
import pyipopt

In [2]:
inputfile = './data_new_volume.csv'
df = pd.read_csv(inputfile)
df['const'] = 1

df['pgmidgrade_km_adj'].fillna(value=10000, inplace=True)

df = df[df['treattype'] < 3]
for elem in df['treattype'].unique():
    df['treat' + str(elem)] = df['treattype'] == elem

for elem in df['choice'].unique():
    df['choice' + str(elem)] = df['choice'] == elem

df['treat1_topusage'] = df['treat1']*df['dv_usageveh_p75p100']
df['treat2_topusage'] = df['treat2']*df['dv_usageveh_p75p100']

df['treat1_college'] = df['treat1']*df['dv_somecollege']
df['treat2_college'] = df['treat2']*df['dv_somecollege']

df.head()

,consumerid,choice,value_total,treattype,pg_km_adj,pgmidgrade_km_adj,pe_km_adj,dv_ctb,dv_bh,dv_rec,...,treat0,treat1,treat2,choice1,choice3,choice2,treat1_topusage,treat2_topusage,treat1_college,treat2_college
0,1,1,93.01,0,0.277658,0.287946,0.278875,0,0,0,...,True,False,False,True,False,False,0,0,0,0
1,2,1,110.23,0,0.273559,0.283694,0.274711,0,0,0,...,True,False,False,True,False,False,0,0,0,0
2,3,1,20.00,0,0.304767,0.316058,0.316189,0,0,0,...,True,False,False,True,False,False,0,0,0,0
3,4,1,60.00,0,0.288213,0.298892,0.303164,0,0,0,...,True,False,False,True,False,False,0,0,0,0
4,5,1,108.88,0,0.247228,0.256388,0.262381,0,0,0,...,True,False,False,True,False,False,0,0,0,0


In [3]:
#df = df[ df['dv_carpriceadj_p75p100'] == 1]
#df = df[ df['dv_usageveh_p75p100'] == 0]
df.head()

,consumerid,choice,value_total,treattype,pg_km_adj,pgmidgrade_km_adj,pe_km_adj,dv_ctb,dv_bh,dv_rec,...,treat0,treat1,treat2,choice1,choice3,choice2,treat1_topusage,treat2_topusage,treat1_college,treat2_college
0,1,1,93.01,0,0.277658,0.287946,0.278875,0,0,0,...,True,False,False,True,False,False,0,0,0,0
1,2,1,110.23,0,0.273559,0.283694,0.274711,0,0,0,...,True,False,False,True,False,False,0,0,0,0
2,3,1,20.00,0,0.304767,0.316058,0.316189,0,0,0,...,True,False,False,True,False,False,0,0,0,0
3,4,1,60.00,0,0.288213,0.298892,0.303164,0,0,0,...,True,False,False,True,False,False,0,0,0,0
4,5,1,108.88,0,0.247228,0.256388,0.262381,0,0,0,...,True,False,False,True,False,False,0,0,0,0


In [4]:
price_labels = ['pg_km_adj', 'pgmidgrade_km_adj', 'pe_km_adj']
value_labels = ['value_total']

# all
Xexpd_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']
Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']

# usage
#Xexpd_labels = ['choice2', 'choice3', 'dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'stationvisit_avgcarprice_adj', 'const']
#Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'stationvisit_avgcarprice_adj', 'const']

# car price
#Xexpd_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']
#Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']

Xlelas_labels = ['const', 'dv_carpriceadj_p75p100', 'dv_somecollege', 'dv_usageveh_p75p100', 'treat1', 'treat2']
Xlsigma_labels = ['const']
Xlmu_labels = ['const']

In [5]:
floatX = 'float64'

In [6]:
choice = df.loc[:, 'choice'].as_matrix() - 1
price  = df.loc[:, price_labels].as_matrix().astype(floatX)
value  = df.loc[:, value_labels].as_matrix().astype(floatX)
Xexpd  = df.loc[:, Xexpd_labels].as_matrix().astype(floatX)
Xutil  = df.loc[:, Xutil_labels].as_matrix().astype(floatX)

Xlelas = df.loc[:, Xlelas_labels].as_matrix().astype(floatX)
Xlsigma = df.loc[:, Xlsigma_labels].as_matrix().astype(floatX)
Xlmu   = df.loc[:, Xlmu_labels].as_matrix().astype(floatX)

In [7]:
nobs, nchoice = price.shape
nobs, nXexpd = Xexpd.shape
nobs, nXutil = Xutil.shape

nobs, nXlelas = Xlelas.shape
nobs, nXlsigma = Xlsigma.shape
nobs, nXlmu = Xlmu.shape

In [8]:
chosenprice = price[range(nobs), choice].reshape(nobs,1)
dvchoice = (choice == np.arange(nchoice).reshape(nchoice,1)).transpose()

In [9]:
theta0 = np.zeros(nXlelas + nXlsigma + nXlmu + nXexpd + (nchoice-1)*nXutil)
theta0[0] = -0.1

In [10]:
def getparams(theta):
    offset = 0
    gammalelas = theta[offset:offset+nXlelas].reshape((nXlelas, 1))
    
    offset += nXlelas
    gammalsigma = theta[offset:offset+nXlsigma].reshape((nXlsigma, 1))
    
    offset += nXlsigma
    gammalmu = theta[offset:offset+nXlmu].reshape((nXlmu, 1))
    
    offset += nXlmu
    betaexpd = theta[offset:offset+nXexpd].reshape((nXexpd, 1))
    
    offset += nXexpd
    betautil = theta[offset:offset+(nchoice-1)*nXutil].reshape((nXutil, nchoice-1))
    
    return gammalelas, gammalsigma, gammalmu, betaexpd, betautil

theta = T.dvector('theta')
gammalelas, gammalsigma, gammalmu, betaexpd, betautil = getparams(theta)

# priceT = theano.shared(price.astype(floatX), name='price')
# pchoiceT = theano.shared(pchoice.astype(floatX), name='pchoice')
# XexpdT = theano.shared(Xexpd.astype(floatX), name='Xexpd')
# XutilT = theano.shared(Xutil.astype(floatX), name='Xutil')
# valueT = theano.shared(value.astype(floatX), name='value')
# XlelasT = theano.shared(Xlelas.astype(floatX), name='Xlelas')
# XlsigmaT = theano.shared(Xlsigma.astype(floatX), name='Xlsigma')
# XlmuT = theano.shared(Xlmu.astype(floatX), name='Xlmu')

mu = T.exp(T.dot(Xlmu, gammalmu))
elas = T.exp(T.dot(Xlelas, gammalelas))
lsigma = T.dot(Xlsigma,gammalsigma)
rho = elas - 1

omegal = 

eta = np.log(value) + rho*np.log(chosenprice) - T.dot(Xexpd,betaexpd)
lexpend = np.log(value) - rho*(np.log(price) - np.log(chosenprice))
util = (T.exp(lexpend)/rho + T.concatenate([np.zeros((nobs,1)), T.dot(Xutil, betautil)],axis=1))/mu
maxutil = T.max(util,axis=1,keepdims=True)
ldenom = maxutil.squeeze() + T.log(T.sum(T.exp(util - maxutil),axis=1))
lprobchoice = T.sum(util*dvchoice.astype(floatX),axis=1) - ldenom
nlogl = (-lprobchoice.sum() +
         T.sum(eta*eta/(2*T.exp(2*lsigma))) + T.sum(lsigma) + nobs*np.log(2*np.pi)/2)

In [11]:
def buildfunc(theta, nloglf):
    return (theano.function([theta], nloglf),
            theano.function([theta], T.grad(nloglf, theta)),
            theano.function([theta], theano.gradient.hessian(nloglf, theta)))

eval_f, eval_grad, eval_hess = buildfunc(theta, nlogl)

In [12]:
eval_hess(theta0)

array([[ -3.43859171e+04,  -8.99090865e+03,  -2.35820871e+04, ...,
          1.02884590e+02,   6.38514758e+00,   3.65833589e+00],
       [ -8.99090865e+03,  -8.99090865e+03,  -7.00792732e+03, ...,
          1.92867888e+01,   1.72682128e+00,   6.70472161e-01],
       [ -2.35820871e+04,  -7.00792732e+03,  -2.35820871e+04, ...,
          8.49467109e+01,   4.07271439e+00,   3.00805382e+00],
       ..., 
       [  1.02884590e+02,   1.92867888e+01,   8.49467109e+01, ...,
          6.78385560e+05,  -5.01137590e+03,   2.33282285e+04],
       [  6.38514758e+00,   1.72682128e+00,   4.07271439e+00, ...,
         -5.01137590e+03,   9.41367893e+02,  -1.73840209e+02],
       [  3.65833589e+00,   6.70472161e-01,   3.00805382e+00, ...,
          2.33282285e+04,  -1.73840209e+02,   8.07026508e+02]])

In [13]:
pyipopt.set_loglevel(1)
thetahat , _, _, _, _, fval = pyipopt.fmin_unconstrained(
    eval_f,
    theta0,
    fprime=eval_grad,
    fhess=eval_hess,
    )


In [14]:
covhat = np.linalg.inv(eval_hess(thetahat))
sehat = np.sqrt(np.diagonal(covhat))
t = thetahat/sehat

gammalelashat, gammalsigmahat, gammalmuhat, betaexpdhat, betautilhat = getparams(thetahat)
gammalelasse, gammalsigmase, gammalmuse, betaexpdse, betautilse = getparams(sehat)
gammalelast, gammalsigmat, gammalmut, betaexpdt, betautilt = getparams(t)

formatstr = '%30s%10.3f%10.3f%10.3f'

print '-'*60

print ' \n*** ln(elas) equation'
for i in range(nXlelas):
    print formatstr % (Xlelas_labels[i], gammalelashat[i], gammalelasse[i], gammalelast[i])
print '-'*60

print ' \n*** ln(sigma) equation'
for i in range(nXlsigma):
    print formatstr % (Xlsigma_labels[i], gammalsigmahat[i], gammalsigmase[i], gammalsigmat[i])
print '-'*60

print ' \n*** ln(mu) equation'
for i in range(nXlmu):
    print formatstr % (Xlmu_labels[i], gammalmuhat[i], gammalmuse[i], gammalmut[i])
print '-'*60

print ' \n*** Expenditure equation'
for i in range(nXexpd):
    print formatstr % (Xexpd_labels[i], betaexpdhat[i], betaexpdse[i], betaexpdt[i])
print '-'*60
    
print ' \n*** Discrete choice equation'
for j in range(nchoice-1):
    print '-------- choice', j+1, '------------------------------------------'
    for i in range(nXutil):
        print formatstr % (Xutil_labels[i], betautilhat[i][j], betautilse[i][j], betautilt[i][j])
print '-'*60


------------------------------------------------------------
 
*** ln(elas) equation
                         const    -0.658     0.203    -3.248
        dv_carpriceadj_p75p100     0.157     0.126     1.246
                dv_somecollege    -0.375     0.191    -1.964
           dv_usageveh_p75p100     0.191     0.145     1.317
                        treat1     0.077     0.025     3.041
                        treat2    -0.006     0.026    -0.221
------------------------------------------------------------
 
*** ln(sigma) equation
                         const    -0.436     0.007   -65.578
------------------------------------------------------------
 
*** ln(mu) equation
                         const     2.351     0.057    41.491
------------------------------------------------------------
 
*** Expenditure equation
                        dv_ctb     0.025     0.018     1.353
                         dv_bh     0.020     0.017     1.174
                        dv_rec    -0.005     0.0